In [ ]:
#| default_exp common._base_model

In [ ]:
#| hide
%load_ext autoreload
%autoreload 2

In [ ]:
#| export
import inspect
import random
import warnings
from contextlib import contextmanager
from copy import deepcopy
from dataclasses import dataclass
from typing import List, Dict, Union, Any, Optional, Tuple

import fsspec
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
import pytorch_lightning as pl
import neuralforecast.losses.pytorch as losses

from neuralforecast.losses.pytorch import BasePointLoss, DistributionLoss
from pytorch_lightning.callbacks.early_stopping import EarlyStopping
from neuralforecast.tsdataset import (
    TimeSeriesDataModule,
    BaseTimeSeriesDataset,
    _DistributedTimeSeriesDataModule,
)
from neuralforecast.common._scalers import TemporalNorm
from neuralforecast.utils import get_indexer_raise_missing

In [ ]:
#| export
@dataclass
class DistributedConfig:
    partitions_path: str
    num_nodes: int
    devices: int

In [ ]:
#| exporti
@contextmanager
def _disable_torch_init():
    """Context manager used to disable pytorch's weight initialization.

    This is especially useful when loading saved models, since when initializing
    a model the weights are also initialized following some method
    (e.g. kaiming uniform), and that time is wasted since we'll override them with
    the saved weights."""
    def noop(*args, **kwargs):
        return
        
    kaiming_uniform = nn.init.kaiming_uniform_
    kaiming_normal = nn.init.kaiming_normal_
    xavier_uniform = nn.init.xavier_uniform_
    xavier_normal = nn.init.xavier_normal_
    
    nn.init.kaiming_uniform_ = noop
    nn.init.kaiming_normal_ = noop
    nn.init.xavier_uniform_ = noop
    nn.init.xavier_normal_ = noop
    try:
        yield
    finally:
        nn.init.kaiming_uniform_ = kaiming_uniform
        nn.init.kaiming_normal_ = kaiming_normal
        nn.init.xavier_uniform_ = xavier_uniform
        nn.init.xavier_normal_ = xavier_normal

In [ ]:
#| exporti
def tensor_to_numpy(tensor: torch.Tensor) -> np.ndarray:
    """Convert a tensor to numpy"""
    if tensor.dtype == torch.bfloat16:
        return tensor.float().numpy()
    
    return tensor.numpy()

In [ ]:
#| export
class BaseModel(pl.LightningModule):
    EXOGENOUS_FUTR = True   # If the model can handle future exogenous variables
    EXOGENOUS_HIST = True   # If the model can handle historical exogenous variables
    EXOGENOUS_STAT = True   # If the model can handle static exogenous variables
    MULTIVARIATE = False    # If the model produces multivariate forecasts (True) or univariate (False)
    RECURRENT = False       # If the model produces forecasts recursively (True) or direct (False)

    def __init__(
        self,
        h: int,
        input_size: int,
        loss: Union[BasePointLoss, DistributionLoss, nn.Module],
        valid_loss: Union[BasePointLoss, DistributionLoss, nn.Module],
        learning_rate: float,
        max_steps: int,
        val_check_steps: int,
        batch_size: int,
        valid_batch_size: Union[int, None],
        windows_batch_size: int,
        inference_windows_batch_size: Union[int, None],
        start_padding_enabled: bool,
        n_series: Union[int, None] = None,
        n_samples: Union[int, None] = 100,
        h_train: int = 1,
        inference_input_size: Union[int, None] = None,
        step_size: int = 1,
        num_lr_decays: int = 0,
        early_stop_patience_steps: int = -1,
        scaler_type: str = 'identity',
        futr_exog_list: Union[List, None] = None,
        hist_exog_list: Union[List, None] = None,
        stat_exog_list: Union[List, None] = None,
        exclude_insample_y: Union[bool, None] = False,
        drop_last_loader: Union[bool, None] = False,
        random_seed: Union[int, None] = 1,
        alias: Union[str, None] = None,
        optimizer: Union[torch.optim.Optimizer, None] = None,
        optimizer_kwargs: Union[Dict, None] = None,
        lr_scheduler: Union[torch.optim.lr_scheduler.LRScheduler, None] = None,
        lr_scheduler_kwargs: Union[Dict, None] = None,
        dataloader_kwargs=None,
        **trainer_kwargs,
    ):
        super().__init__()

        # Multivarariate checks
        if self.MULTIVARIATE and n_series is None:
            raise Exception(f'{type(self).__name__} is a multivariate model. Please set n_series to the number of unique time series in your dataset.')
        if not self.MULTIVARIATE:
            n_series = 1
        self.n_series = n_series          

        # Protections for previous recurrent models
        if input_size < 1:
            input_size = 3 * h
            warnings.warn(
                f'Input size too small. Automatically setting input size to 3 * horizon = {input_size}'
            )

        if inference_input_size is None:
            inference_input_size = input_size            
        elif inference_input_size is not None and inference_input_size < 1:
            inference_input_size = input_size
            warnings.warn(
                f'Inference input size too small. Automatically setting inference input size to input_size = {input_size}'
            )

        # For recurrent models we need one additional input as we need to shift insample_y to use it as input
        if self.RECURRENT:
            input_size += 1
            inference_input_size += 1

        # Attributes needed for recurrent models
        self.horizon_backup = h
        self.input_size_backup = input_size
        self.n_samples = n_samples
        if self.RECURRENT:
            if (
                hasattr(loss, "horizon_weight")
                and loss.horizon_weight is not None
                and h_train != h
            ):
                warnings.warn(
                    f"Setting h_train={h} to match the horizon_weight length."
                )
                h_train = h
            self.h_train = h_train
            self.inference_input_size = inference_input_size
            self.rnn_state = None
            self.maintain_state = False

        with warnings.catch_warnings(record=False):
            warnings.filterwarnings('ignore')
            # the following line issues a warning about the loss attribute being saved
            # but we do want to save it
            self.save_hyperparameters() # Allows instantiation from a checkpoint from class
        self.random_seed = random_seed
        pl.seed_everything(self.random_seed, workers=True)

        # Loss
        self.loss = loss
        if valid_loss is None:
            self.valid_loss = loss
        else:
            self.valid_loss = valid_loss
        self.train_trajectories: List = []
        self.valid_trajectories: List = []

        # Optimization
        if optimizer is not None and not issubclass(optimizer, torch.optim.Optimizer):
            raise TypeError("optimizer is not a valid subclass of torch.optim.Optimizer")
        self.optimizer = optimizer
        self.optimizer_kwargs = optimizer_kwargs if optimizer_kwargs is not None else {}

        # lr scheduler
        if lr_scheduler is not None and not issubclass(lr_scheduler, torch.optim.lr_scheduler.LRScheduler):
            raise TypeError("lr_scheduler is not a valid subclass of torch.optim.lr_scheduler.LRScheduler")
        self.lr_scheduler = lr_scheduler
        self.lr_scheduler_kwargs = lr_scheduler_kwargs if lr_scheduler_kwargs is not None else {}

        # Variables
        self.futr_exog_list = list(futr_exog_list) if futr_exog_list is not None else []
        self.hist_exog_list = list(hist_exog_list) if hist_exog_list is not None else []
        self.stat_exog_list = list(stat_exog_list) if stat_exog_list is not None else []

        # Set data sizes
        self.futr_exog_size = len(self.futr_exog_list)
        self.hist_exog_size = len(self.hist_exog_list)
        self.stat_exog_size = len(self.stat_exog_list)   

        # Check if model supports exogenous, otherwise raise Exception
        if not self.EXOGENOUS_FUTR and self.futr_exog_size > 0:
            raise Exception(f'{type(self).__name__} does not support future exogenous variables.')
        if not self.EXOGENOUS_HIST and self.hist_exog_size > 0:
            raise Exception(f'{type(self).__name__} does not support historical exogenous variables.')
        if not self.EXOGENOUS_STAT and self.stat_exog_size > 0:
            raise Exception(f'{type(self).__name__} does not support static exogenous variables.')

        # Protections for loss functions
        if isinstance(self.loss, (losses.IQLoss, losses.HuberIQLoss)):
            loss_type = type(self.loss)
            if not isinstance(self.valid_loss, loss_type):
                raise Exception(f'Please set valid_loss={type(self.loss).__name__}() when training with {type(self.loss).__name__}')
        if isinstance(self.loss, (losses.MQLoss, losses.HuberMQLoss)):
            if not isinstance(self.valid_loss, (losses.MQLoss, losses.HuberMQLoss)):
                raise Exception(f'Please set valid_loss to MQLoss() or HuberMQLoss() when training with {type(self.loss).__name__}')
        if isinstance(self.valid_loss, (losses.IQLoss, losses.HuberIQLoss)):
            valid_loss_type = type(self.valid_loss)
            if not isinstance(self.loss, valid_loss_type):
                raise Exception(f'Please set loss={type(self.valid_loss).__name__}() when validating with {type(self.valid_loss).__name__}')        

        # Deny impossible loss / valid_loss combinations
        if isinstance(self.loss, losses.BasePointLoss) and self.valid_loss.is_distribution_output:
            raise Exception(f'Validation with distribution loss {type(self.valid_loss).__name__} is not possible when using loss={type(self.loss).__name__}. Please use a point valid_loss (MAE, MSE, ...)')
        elif self.valid_loss.is_distribution_output and self.valid_loss is not loss:
            # Maybe we should raise a Warning or an Exception here, but meh for now.
            self.valid_loss = loss
        
        if isinstance(self.loss, (losses.relMSE, losses.Accuracy, losses.sCRPS)):
            raise Exception(f"{type(self.loss).__name__} cannot be used for training. Please use another loss function (MAE, MSE, ...)")
        
        if isinstance(self.valid_loss, (losses.relMSE)):
            raise Exception(f"{type(self.valid_loss).__name__} cannot be used for validation. Please use another valid_loss (MAE, MSE, ...)")

        ## Trainer arguments ##
        # Max steps, validation steps and check_val_every_n_epoch
        trainer_kwargs = {**trainer_kwargs, 'max_steps': max_steps}

        if 'max_epochs' in trainer_kwargs.keys():
            raise Exception('max_epochs is deprecated, use max_steps instead.')

        # Callbacks
        if early_stop_patience_steps > 0:
            if 'callbacks' not in trainer_kwargs:
                trainer_kwargs['callbacks'] = []
            trainer_kwargs['callbacks'].append(
                EarlyStopping(
                    monitor='ptl/val_loss', patience=early_stop_patience_steps
                )
            )

        # Add GPU accelerator if available
        if trainer_kwargs.get('accelerator', None) is None:
            if torch.cuda.is_available():
                trainer_kwargs['accelerator'] = "gpu"
        if trainer_kwargs.get('devices', None) is None:
            if torch.cuda.is_available():
                trainer_kwargs['devices'] = -1

        # Avoid saturating local memory, disabled fit model checkpoints
        if trainer_kwargs.get('enable_checkpointing', None) is None:
            trainer_kwargs['enable_checkpointing'] = False

        # Set other attributes
        self.trainer_kwargs = trainer_kwargs
        self.h = h
        self.input_size = input_size
        self.windows_batch_size = windows_batch_size
        self.start_padding_enabled = start_padding_enabled

        # Padder to complete train windows, 
        # example y=[1,2,3,4,5] h=3 -> last y_output = [5,0,0]
        if start_padding_enabled:
            self.padder_train = nn.ConstantPad1d(padding=(self.input_size-1, self.h), value=0.0)
        else:
            self.padder_train = nn.ConstantPad1d(padding=(0, self.h), value=0.0)

        # Batch sizes
        if self.MULTIVARIATE and n_series is not None:
            self.batch_size = max(batch_size, n_series)
            if valid_batch_size is not None:
                valid_batch_size = max(valid_batch_size, n_series)
        else:
            self.batch_size = batch_size
        
        if valid_batch_size is None:
            self.valid_batch_size = self.batch_size
        else:
            self.valid_batch_size = valid_batch_size

        if inference_windows_batch_size is None:
            self.inference_windows_batch_size = windows_batch_size
        else:
            self.inference_windows_batch_size = inference_windows_batch_size

        # Optimization 
        self.learning_rate = learning_rate
        self.max_steps = max_steps
        self.num_lr_decays = num_lr_decays
        self.lr_decay_steps = (
            max(max_steps // self.num_lr_decays, 1) if self.num_lr_decays > 0 else 10e7
        )
        self.early_stop_patience_steps = early_stop_patience_steps
        self.val_check_steps = val_check_steps
        self.windows_batch_size = windows_batch_size
        self.step_size = step_size
        
        # If the model does not support exogenous, it can't support exclude_insample_y
        if exclude_insample_y and not (self.EXOGENOUS_FUTR or self.EXOGENOUS_HIST or self.EXOGENOUS_STAT):
            raise Exception(f'{type(self).__name__} does not support `exclude_insample_y=True`. Please set `exclude_insample_y=False`')

        self.exclude_insample_y = exclude_insample_y

        # Scaler
        self.scaler = TemporalNorm(
            scaler_type=scaler_type,
            dim=1,  # Time dimension is 1.
            num_features= 1 + len(self.hist_exog_list) + len(self.futr_exog_list)
        )

        # Fit arguments
        self.val_size = 0
        self.test_size = 0

        # Model state
        self.decompose_forecast = False

        # DataModule arguments
        self.dataloader_kwargs = dataloader_kwargs
        self.drop_last_loader = drop_last_loader
        # used by on_validation_epoch_end hook
        self.validation_step_outputs: List = []
        self.alias = alias

    def __repr__(self):
        return type(self).__name__ if self.alias is None else self.alias

    def _check_exog(self, dataset):
        temporal_cols = set(dataset.temporal_cols.tolist())
        static_cols = set(dataset.static_cols.tolist() if dataset.static_cols is not None else [])

        missing_hist = set(self.hist_exog_list) - temporal_cols
        missing_futr = set(self.futr_exog_list) - temporal_cols
        missing_stat = set(self.stat_exog_list) - static_cols
        if missing_hist:
            raise Exception(f'{missing_hist} historical exogenous variables not found in input dataset')
        if missing_futr:
            raise Exception(f'{missing_futr} future exogenous variables not found in input dataset')
        if missing_stat:
            raise Exception(f'{missing_stat} static exogenous variables not found in input dataset')

    def _restart_seed(self, random_seed):
        if random_seed is None:
            random_seed = self.random_seed
        torch.manual_seed(random_seed)

    def _get_temporal_exogenous_cols(self, temporal_cols):
        return list(
            set(temporal_cols.tolist()) & set(self.hist_exog_list + self.futr_exog_list)
        )
    
    def _set_quantiles(self, quantiles=None):
        if quantiles is None and isinstance(self.loss, (losses.IQLoss, losses.HuberIQLoss)):
            self.loss.update_quantile(q=[0.5])
        elif hasattr(self.loss, 'update_quantile') and callable(self.loss.update_quantile):
            self.loss.update_quantile(q=quantiles)

    def _fit_distributed(
        self,
        distributed_config,
        datamodule,
        val_size,
        test_size,
    ):
        assert distributed_config is not None
        from pyspark.ml.torch.distributor import TorchDistributor

        def train_fn(
            model_cls,
            model_params,
            datamodule,
            trainer_kwargs,
            num_tasks,
            num_proc_per_task,
            val_size,
            test_size,
        ):
            import pytorch_lightning as pl

            # we instantiate here to avoid pickling large tensors (weights)
            model = model_cls(**model_params)
            model.val_size = val_size
            model.test_size = test_size
            for arg in ('devices', 'num_nodes'):
                trainer_kwargs.pop(arg, None)
            trainer = pl.Trainer(
                strategy="ddp",
                use_distributed_sampler=False,  # to ensure our dataloaders are used as-is
                num_nodes=num_tasks,
                devices=num_proc_per_task,
                **trainer_kwargs,
            )
            trainer.fit(model=model, datamodule=datamodule)
            model.metrics = trainer.callback_metrics
            model.__dict__.pop('_trainer', None)
            return model

        def is_gpu_accelerator(accelerator):
            from pytorch_lightning.accelerators.cuda import CUDAAccelerator

            return (
                accelerator == "gpu"
                or isinstance(accelerator, CUDAAccelerator)
                or (accelerator == "auto" and CUDAAccelerator.is_available())
            )

        local_mode = distributed_config.num_nodes == 1
        if local_mode:
            num_tasks = 1
            num_proc_per_task = distributed_config.devices
        else:
            num_tasks = distributed_config.num_nodes * distributed_config.devices
            num_proc_per_task = 1  # number of GPUs per task
        num_proc = num_tasks * num_proc_per_task
        use_gpu = is_gpu_accelerator(self.trainer_kwargs["accelerator"])
        model = TorchDistributor(
            num_processes=num_proc,
            local_mode=local_mode,
            use_gpu=use_gpu,
        ).run(
            train_fn,
            model_cls=type(self),
            model_params=self.hparams,
            datamodule=datamodule,
            trainer_kwargs=self.trainer_kwargs,
            num_tasks=num_tasks,
            num_proc_per_task=num_proc_per_task,
            val_size=val_size,
            test_size=test_size,
        )
        return model

    def _fit(
        self,
        dataset,
        batch_size,
        valid_batch_size=1024,
        val_size=0,
        test_size=0,
        random_seed=None,
        shuffle_train=True,
        distributed_config=None,
    ):
        self._check_exog(dataset)
        self._restart_seed(random_seed)

        self.val_size = val_size
        self.test_size = test_size
        is_local = isinstance(dataset, BaseTimeSeriesDataset)
        if is_local:
            datamodule_constructor = TimeSeriesDataModule
        else:
            datamodule_constructor = _DistributedTimeSeriesDataModule
        
        dataloader_kwargs = self.dataloader_kwargs if self.dataloader_kwargs is not None else {}
        datamodule = datamodule_constructor(
            dataset=dataset, 
            batch_size=batch_size,
            valid_batch_size=valid_batch_size,
            drop_last=self.drop_last_loader,
            shuffle_train=shuffle_train,
            **dataloader_kwargs
        )

        if self.val_check_steps > self.max_steps:
            warnings.warn(
                'val_check_steps is greater than max_steps, '
                'setting val_check_steps to max_steps.'
            )
        val_check_interval = min(self.val_check_steps, self.max_steps)
        self.trainer_kwargs['val_check_interval'] = int(val_check_interval)
        self.trainer_kwargs['check_val_every_n_epoch'] = None

        if is_local:
            model = self
            trainer = pl.Trainer(**model.trainer_kwargs)
            trainer.fit(model, datamodule=datamodule)
            model.metrics = trainer.callback_metrics
            model.__dict__.pop('_trainer', None)
        else:
            model = self._fit_distributed(
                distributed_config,
                datamodule,
                val_size,
                test_size,
            )
        return model

    def _extract_target_features(self, dataset_batch: Dict) -> np.ndarray:
        """
        Extract target, historical exogenous, future exogenous, and static exogenous features for SHAP explanation.
        """
        # TimeSeriesDataset returns temporal data as [batch_size, features, time_steps]  
        temporal_data = dataset_batch['temporal']  # Shape: [batch_size, features, time_steps]
        temporal_cols = dataset_batch['temporal_cols']  # Column names
        
        # Check what exogenous features the model supports
        has_hist_exog = (hasattr(self, 'EXOGENOUS_HIST') and self.EXOGENOUS_HIST and 
                        hasattr(self, 'hist_exog_list') and self.hist_exog_list)
        has_futr_exog = (hasattr(self, 'EXOGENOUS_FUTR') and self.EXOGENOUS_FUTR and 
                        hasattr(self, 'futr_exog_list') and self.futr_exog_list)
        has_stat_exog = (hasattr(self, 'EXOGENOUS_STAT') and self.EXOGENOUS_STAT and 
                        hasattr(self, 'stat_exog_list') and self.stat_exog_list)
        
        if self.MULTIVARIATE:
            # For multivariate models, extract n_series worth of data
            n_series = getattr(self, 'n_series', 1)
            
            # Take the first n_series from temporal_data (assumes target series come first)
            if temporal_data.shape[1] >= n_series:
                series_data = temporal_data[:, :n_series, :]  # [batch_size, n_series, time_steps]
            else:
                # If we don't have enough series, pad or repeat
                series_data = temporal_data[:, :1, :].repeat(1, n_series, 1)
                print(f"Warning: Dataset has {temporal_data.shape[1]} series but model expects {n_series}. Using repeated data.")
            
            # Take the last input_size time steps for all series
            if series_data.shape[2] >= self.input_size:
                target_features = series_data[:, :, -self.input_size:]  # [batch_size, n_series, input_size]
            else:
                # Pad with zeros if not enough history
                pad_size = self.input_size - series_data.shape[2] 
                target_features = F.pad(series_data, (pad_size, 0), value=0.0)
            
            # Flatten to [batch_size, n_series * input_size] for SHAP
            batch_size, n_series_actual, input_size = target_features.shape
            target_features = target_features.reshape(batch_size, n_series_actual * input_size)
            
            # Add multivariate historical exogenous support
            if has_hist_exog:
                hist_exog_features = []
                
                # Find indices of historical exogenous columns
                for exog_col in self.hist_exog_list:
                    if exog_col in temporal_cols:
                        exog_idx = temporal_cols.get_loc(exog_col)
                        exog_data = temporal_data[:, exog_idx, :]  # [batch_size, time_steps]
                        
                        # Take the last input_size time steps for this exogenous feature
                        if exog_data.shape[1] >= self.input_size:
                            exog_features = exog_data[:, -self.input_size:]  # [batch_size, input_size]
                        else:
                            # Pad with zeros if not enough history
                            pad_size = self.input_size - exog_data.shape[1]
                            exog_features = F.pad(exog_data, (pad_size, 0), value=0.0)
                        
                        hist_exog_features.append(exog_features)
                    else:
                        print(f"Warning: Historical exogenous feature '{exog_col}' not found in dataset columns.")
                        # Create zero features as placeholder
                        zero_features = torch.zeros(target_features.shape[0], self.input_size)
                        hist_exog_features.append(zero_features)
                
                # Concatenate historical exogenous features
                if hist_exog_features:
                    all_hist_exog = torch.stack(hist_exog_features, dim=1)  # [batch_size, n_hist_exog, input_size]
                    all_hist_exog = all_hist_exog.reshape(target_features.shape[0], -1)  # [batch_size, n_hist_exog * input_size]
                    target_features = torch.cat([target_features, all_hist_exog], dim=1)  # [batch_size, n_series * input_size + n_hist_exog * input_size]
            
        else:
            # For univariate models, extract target series and exogenous features
            y_idx = dataset_batch['y_idx']  # Index of target variable in features
            target_data = temporal_data[:, y_idx, :]  # Shape: [batch_size, time_steps]
            
            # Take the last input_size time steps for target
            if target_data.shape[1] >= self.input_size:
                target_features = target_data[:, -self.input_size:]  # Shape: [batch_size, input_size]
            else:
                # Pad with zeros if not enough history
                pad_size = self.input_size - target_data.shape[1]
                target_features = F.pad(target_data, (pad_size, 0), value=0.0)
            
            # Extract historical exogenous features if model supports them
            if has_hist_exog:
                hist_exog_features = []
                
                # Find indices of historical exogenous columns
                for exog_col in self.hist_exog_list:
                    if exog_col in temporal_cols:
                        exog_idx = temporal_cols.get_loc(exog_col)
                        exog_data = temporal_data[:, exog_idx, :]  # [batch_size, time_steps]
                        
                        # Take the last input_size time steps for this exogenous feature
                        if exog_data.shape[1] >= self.input_size:
                            exog_features = exog_data[:, -self.input_size:]  # [batch_size, input_size]
                        else:
                            # Pad with zeros if not enough history
                            pad_size = self.input_size - exog_data.shape[1]
                            exog_features = F.pad(exog_data, (pad_size, 0), value=0.0)
                        
                        hist_exog_features.append(exog_features)
                    else:
                        print(f"Warning: Historical exogenous feature '{exog_col}' not found in dataset columns.")
                        # Create zero features as placeholder
                        zero_features = torch.zeros(target_features.shape[0], self.input_size)
                        hist_exog_features.append(zero_features)
                
                # Concatenate target and historical exogenous features
                if hist_exog_features:
                    all_hist_exog = torch.stack(hist_exog_features, dim=1)  # [batch_size, n_hist_exog, input_size]
                    all_hist_exog = all_hist_exog.reshape(target_features.shape[0], -1)  # [batch_size, n_hist_exog * input_size]
                    target_features = torch.cat([target_features, all_hist_exog], dim=1)  # [batch_size, input_size + n_hist_exog * input_size]
        
        # Extract future exogenous features if model supports them (both univariate and multivariate)
        if has_futr_exog:
            futr_exog_features = []
            horizon = getattr(self, 'h', 1)  # Forecast horizon
            
            # For NHITS, future exogenous needs input_size + horizon time steps
            # For other models, just horizon time steps
            if self.__class__.__name__ == 'NHITS':
                futr_exog_time_steps = self.input_size + horizon
            else:
                futr_exog_time_steps = horizon
            
            # Find indices of future exogenous columns
            for exog_col in self.futr_exog_list:
                if exog_col in temporal_cols:
                    exog_idx = temporal_cols.get_loc(exog_col)
                    exog_data = temporal_data[:, exog_idx, :]  # [batch_size, time_steps]
                    
                    # Take the required time steps for future exogenous
                    if exog_data.shape[1] >= futr_exog_time_steps:
                        futr_exog_data = exog_data[:, -futr_exog_time_steps:]  # [batch_size, required_steps]
                    else:
                        # Pad with zeros if not enough future data
                        pad_size = futr_exog_time_steps - exog_data.shape[1]
                        futr_exog_data = F.pad(exog_data, (0, pad_size), value=0.0)[:, -futr_exog_time_steps:]
                    
                    futr_exog_features.append(futr_exog_data)
                else:
                    print(f"Warning: Future exogenous feature '{exog_col}' not found in dataset columns.")
                    # Create zero features as placeholder
                    zero_features = torch.zeros(target_features.shape[0], futr_exog_time_steps)
                    futr_exog_features.append(zero_features)
            
            # Concatenate future exogenous features
            if futr_exog_features:
                all_futr_exog = torch.stack(futr_exog_features, dim=1)  # [batch_size, n_futr_exog, required_steps]
                all_futr_exog = all_futr_exog.reshape(target_features.shape[0], -1)  # [batch_size, n_futr_exog * required_steps]
                target_features = torch.cat([target_features, all_futr_exog], dim=1)  # [batch_size, prev_features + n_futr_exog * required_steps]
        
        # Extract static exogenous features if model supports them (both univariate and multivariate)
        if has_stat_exog:
            # Static exogenous data is in dataset_batch['static']
            if 'static' in dataset_batch and dataset_batch['static'] is not None:
                static_data = dataset_batch['static']  # Shape: [batch_size, n_static_features]
                
                if self.MULTIVARIATE:
                    # For multivariate models like TSMixerx, static exog is expected as [n_series, n_static_features]
                    # We need to repeat static features across series or take first n_series rows
                    n_series = getattr(self, 'n_series', 1)
                    n_stat_features = static_data.shape[1]
                    
                    if static_data.shape[0] >= n_series:
                        # Take first n_series rows
                        static_features = static_data[:n_series, :]  # [n_series, n_static_features]
                    else:
                        # Repeat first row across series
                        static_features = static_data[:1, :].repeat(n_series, 1)  # [n_series, n_static_features]
                    
                    # Flatten for SHAP: [n_series, n_static_features] -> [n_series * n_static_features]
                    static_features = static_features.reshape(-1)  # [n_series * n_static_features]
                    target_features = torch.cat([target_features, static_features.unsqueeze(0).repeat(target_features.shape[0], 1)], dim=1)
                else:
                    # For univariate models like NHITS, static exog is expected as [batch_size, n_static_features]
                    static_features = static_data  # [batch_size, n_static_features]
                    target_features = torch.cat([target_features, static_features], dim=1)  # [batch_size, prev_features + n_static_features]
            else:
                print("Warning: Model expects static exogenous features but none found in dataset batch.")
                # Create zero static features as placeholder
                n_stat_features = len(self.stat_exog_list)
                
                if self.MULTIVARIATE:
                    n_series = getattr(self, 'n_series', 1)
                    zero_static = torch.zeros(target_features.shape[0], n_series * n_stat_features)
                else:
                    zero_static = torch.zeros(target_features.shape[0], n_stat_features)
                
                target_features = torch.cat([target_features, zero_static], dim=1)
        
        return target_features.cpu().numpy() if torch.is_tensor(target_features) else target_features

    def _get_feature_names(self) -> List[str]:
        """
        Get feature names for SHAP plots (with support for all exogenous types).
        """
        feature_names = []
        
        if self.MULTIVARIATE:
            # For multivariate models, create names for all series and their lags
            n_series = getattr(self, 'n_series', 1)
            
            for series_idx in range(n_series):
                for lag_idx in range(self.input_size):
                    feature_names.append(f"series_{series_idx}_lag_{lag_idx + 1}")
            
            # Add multivariate historical exogenous feature names
            has_hist_exog = (hasattr(self, 'EXOGENOUS_HIST') and self.EXOGENOUS_HIST and 
                            hasattr(self, 'hist_exog_list') and self.hist_exog_list)
            
            if has_hist_exog:
                for exog_col in self.hist_exog_list:
                    for lag_idx in range(self.input_size):
                        feature_names.append(f"{exog_col}_lag_{lag_idx + 1}")
            
        else:
            # For univariate models, add target lags
            for lag_idx in range(self.input_size):
                feature_names.append(f"y_lag_{lag_idx + 1}")
            
            # Add historical exogenous feature names if model supports them
            has_hist_exog = (hasattr(self, 'EXOGENOUS_HIST') and self.EXOGENOUS_HIST and 
                            hasattr(self, 'hist_exog_list') and self.hist_exog_list)
            
            if has_hist_exog:
                for exog_col in self.hist_exog_list:
                    for lag_idx in range(self.input_size):
                        feature_names.append(f"{exog_col}_lag_{lag_idx + 1}")
        
        # Add future exogenous feature names (both univariate and multivariate)
        has_futr_exog = (hasattr(self, 'EXOGENOUS_FUTR') and self.EXOGENOUS_FUTR and 
                        hasattr(self, 'futr_exog_list') and self.futr_exog_list)
        
        if has_futr_exog:
            horizon = getattr(self, 'h', 1)
            
            # For NHITS, future exogenous spans input_size + horizon time steps
            # For other models, just horizon time steps
            if self.__class__.__name__ == 'NHITS':
                futr_exog_time_steps = self.input_size + horizon
                time_step_prefix = "step"  # step_1, step_2, etc. (includes both historical and future periods)
            else:
                futr_exog_time_steps = horizon
                time_step_prefix = "step"  # step_1, step_2, etc. (forecast period only)
            
            for exog_col in self.futr_exog_list:
                for step_idx in range(futr_exog_time_steps):
                    feature_names.append(f"{exog_col}_{time_step_prefix}_{step_idx + 1}")
        
        # Add static exogenous feature names (both univariate and multivariate)
        has_stat_exog = (hasattr(self, 'EXOGENOUS_STAT') and self.EXOGENOUS_STAT and 
                        hasattr(self, 'stat_exog_list') and self.stat_exog_list)
        
        if has_stat_exog:
            if self.MULTIVARIATE:
                # For multivariate models, static features are per series
                n_series = getattr(self, 'n_series', 1)
                for series_idx in range(n_series):
                    for stat_col in self.stat_exog_list:
                        feature_names.append(f"{stat_col}_series_{series_idx}_static")
            else:
                # For univariate models, static features are global
                for stat_col in self.stat_exog_list:
                    feature_names.append(f"{stat_col}_static")
        
        return feature_names

    def _parse_shap_input(self, X_flat: np.ndarray) -> Dict[str, torch.Tensor]:
        """
        Parse flattened SHAP input back into model's expected format.
        """
        # Convert to tensor
        X_tensor = torch.FloatTensor(X_flat)
        
        # Check what exogenous features the model supports
        has_hist_exog = (hasattr(self, 'EXOGENOUS_HIST') and self.EXOGENOUS_HIST and 
                        hasattr(self, 'hist_exog_list') and self.hist_exog_list)
        has_futr_exog = (hasattr(self, 'EXOGENOUS_FUTR') and self.EXOGENOUS_FUTR and 
                        hasattr(self, 'futr_exog_list') and self.futr_exog_list)
        has_stat_exog = (hasattr(self, 'EXOGENOUS_STAT') and self.EXOGENOUS_STAT and 
                        hasattr(self, 'stat_exog_list') and self.stat_exog_list)
        
        if self.MULTIVARIATE:
            # For multivariate models: handle target + historical exog + future exog + static exog
            batch_size = X_tensor.shape[0]
            n_series = getattr(self, 'n_series', 1)
            
            # Calculate sizes
            target_size = n_series * self.input_size
            hist_exog_size = 0
            futr_exog_size = 0
            stat_exog_size = 0
            
            if has_hist_exog:
                hist_exog_size = len(self.hist_exog_list) * self.input_size
            if has_futr_exog:
                horizon = getattr(self, 'h', 1)
                # For NHITS, future exogenous spans input_size + horizon time steps
                if self.__class__.__name__ == 'NHITS':
                    futr_exog_size = len(self.futr_exog_list) * (self.input_size + horizon)
                else:
                    futr_exog_size = len(self.futr_exog_list) * horizon
            if has_stat_exog:
                stat_exog_size = len(self.stat_exog_list) * n_series  # For multivariate: n_stat_features * n_series
            
            # Split features
            current_idx = 0
            
            # Target series data
            target_flat = X_tensor[:, current_idx:current_idx + target_size]
            current_idx += target_size
            
            # Reshape target: [batch_size, n_series * input_size] -> [batch_size, input_size, n_series]
            insample_y = target_flat.reshape(batch_size, n_series, self.input_size)
            insample_y = insample_y.transpose(1, 2)  # [batch_size, input_size, n_series]
            
            # Handle historical exogenous (TSMixerx format)
            if has_hist_exog and hist_exog_size > 0:
                hist_exog_flat = X_tensor[:, current_idx:current_idx + hist_exog_size]
                current_idx += hist_exog_size
                
                # TSMixerx expects hist_exog as [batch_size, hist_exog_size, input_size, n_series]
                n_hist_exog = len(self.hist_exog_list)
                
                # Reshape: [batch_size, n_hist_exog * input_size] -> [batch_size, n_hist_exog, input_size]
                hist_exog = hist_exog_flat.reshape(batch_size, n_hist_exog, self.input_size)
                
                # Add n_series dimension: [batch_size, n_hist_exog, input_size] -> [batch_size, n_hist_exog, input_size, n_series]
                # For multivariate, hist_exog features are typically repeated across series
                hist_exog = hist_exog.unsqueeze(-1).repeat(1, 1, 1, n_series)
            else:
                hist_exog = None
            
            # Handle future exogenous for multivariate models (TSMixerx format)
            if has_futr_exog and futr_exog_size > 0:
                futr_exog_flat = X_tensor[:, current_idx:current_idx + futr_exog_size]
                current_idx += futr_exog_size
                n_futr_exog = len(self.futr_exog_list)
                horizon = getattr(self, 'h', 1)
                
                if self.__class__.__name__ == 'NHITS':
                    # NHITS (though it's univariate) would need input_size + horizon
                    futr_exog_time_steps = self.input_size + horizon
                else:
                    # TSMixerx and other multivariate models expect just horizon
                    futr_exog_time_steps = horizon
                
                # TSMixerx expects futr_exog as [batch_size, futr_exog_size, input_size + horizon, n_series]
                # Reshape: [batch_size, n_futr_exog * time_steps] -> [batch_size, n_futr_exog, time_steps]
                futr_exog = futr_exog_flat.reshape(batch_size, n_futr_exog, futr_exog_time_steps)
                
                if self.__class__.__name__ == 'TSMixerx':
                    # For TSMixerx, we need to pad with input_size zeros at the beginning
                    # since it expects [input_size + horizon] but we only have [horizon] from SHAP
                    if futr_exog_time_steps == horizon:  # Only horizon provided
                        zeros_pad = torch.zeros(batch_size, n_futr_exog, self.input_size)
                        futr_exog = torch.cat([zeros_pad, futr_exog], dim=2)  # [batch_size, n_futr_exog, input_size + horizon]
                    
                    # Add n_series dimension: [batch_size, n_futr_exog, input_size + horizon] -> [batch_size, n_futr_exog, input_size + horizon, n_series]
                    futr_exog = futr_exog.unsqueeze(-1).repeat(1, 1, 1, n_series)
                else:
                    # For other multivariate models, transpose to [batch_size, time_steps, n_futr_exog]
                    futr_exog = futr_exog.transpose(1, 2)
            else:
                futr_exog = None
            
            # Handle static exogenous for multivariate models
            if has_stat_exog and stat_exog_size > 0:
                stat_exog_flat = X_tensor[:, current_idx:current_idx + stat_exog_size]
                # For TSMixerx: reshape [batch_size, n_series * n_stat_features] -> [n_series, n_stat_features]
                n_stat_features = len(self.stat_exog_list)
                stat_exog = stat_exog_flat[0].reshape(n_series, n_stat_features)  # Take first batch sample
            else:
                stat_exog = None
            
            # For multivariate models, no insample_mask needed
            insample_mask = None
            
        else:
            # For univariate models: handle target + historical exog + future exog + static exog
            batch_size = X_tensor.shape[0]
            
            # Calculate sizes
            target_size = self.input_size
            hist_exog_size = 0
            futr_exog_size = 0
            stat_exog_size = 0
            
            if has_hist_exog:
                hist_exog_size = len(self.hist_exog_list) * self.input_size
            if has_futr_exog:
                horizon = getattr(self, 'h', 1)
                # For NHITS, future exogenous spans input_size + horizon time steps
                if self.__class__.__name__ == 'NHITS':
                    futr_exog_size = len(self.futr_exog_list) * (self.input_size + horizon)
                else:
                    futr_exog_size = len(self.futr_exog_list) * horizon
            if has_stat_exog:
                stat_exog_size = len(self.stat_exog_list)
            
            # Split the flattened input
            current_idx = 0
            
            # Target features
            target_flat = X_tensor[:, current_idx:current_idx + target_size]
            current_idx += target_size
            
            # Historical exogenous features
            if has_hist_exog and hist_exog_size > 0:
                hist_exog_flat = X_tensor[:, current_idx:current_idx + hist_exog_size]
                current_idx += hist_exog_size
                
                # Reshape: [batch_size, n_hist_exog * input_size] -> [batch_size, input_size, n_hist_exog]
                n_hist_exog = len(self.hist_exog_list)
                hist_exog = hist_exog_flat.reshape(batch_size, n_hist_exog, self.input_size)
                hist_exog = hist_exog.transpose(1, 2)
            else:
                hist_exog = None
            
            # Future exogenous features
            if has_futr_exog and futr_exog_size > 0:
                futr_exog_flat = X_tensor[:, current_idx:current_idx + futr_exog_size]
                current_idx += futr_exog_size
                n_futr_exog = len(self.futr_exog_list)
                
                # Handle different model requirements for future exogenous
                if self.__class__.__name__ == 'NHITS':
                    # NHITS expects [batch_size, input_size + horizon, n_futr_exog]
                    horizon = getattr(self, 'h', 1)
                    futr_exog_time_steps = self.input_size + horizon
                    
                    # Reshape: [batch_size, n_futr_exog * (input_size + horizon)] -> [batch_size, input_size + horizon, n_futr_exog]
                    futr_exog = futr_exog_flat.reshape(batch_size, n_futr_exog, futr_exog_time_steps)
                    futr_exog = futr_exog.transpose(1, 2)
                else:
                    # Other models expect [batch_size, horizon, n_futr_exog]
                    horizon = getattr(self, 'h', 1)
                    
                    futr_exog = futr_exog_flat.reshape(batch_size, n_futr_exog, horizon)
                    futr_exog = futr_exog.transpose(1, 2)
            else:
                futr_exog = None
            
            # Static exogenous features
            if has_stat_exog and stat_exog_size > 0:
                stat_exog_flat = X_tensor[:, current_idx:current_idx + stat_exog_size]
                # Static exog is used as-is: [batch_size, n_stat_exog]
                stat_exog = stat_exog_flat
                print(f"DEBUG: Static exog shape: {stat_exog.shape}")
            else:
                stat_exog = None
            
            # Reshape target: [batch_size, input_size] -> [batch_size, input_size, 1]
            insample_y = target_flat.unsqueeze(-1)
            
            # Create insample_mask based on non-zero values (for models like NHITS)
            insample_mask = (target_flat != 0.0).float().unsqueeze(-1)
        
        # Move to same device as model
        device = next(self.parameters()).device
        insample_y = insample_y.to(device)
        if insample_mask is not None:
            insample_mask = insample_mask.to(device)     
        if hist_exog is not None:
            hist_exog = hist_exog.to(device)
        if futr_exog is not None:
            futr_exog = futr_exog.to(device)
        if stat_exog is not None:
            stat_exog = stat_exog.to(device)
        
        # Return dictionary matching model's forward method expected windows_batch format
        windows_batch = {
            'insample_y': insample_y,
            'hist_exog': hist_exog,
            'futr_exog': futr_exog,
            'stat_exog': stat_exog   # Now includes actual static exogenous data
        }
        
        # Add insample_mask only for univariate models that might need it
        if not self.MULTIVARIATE:
            windows_batch['insample_mask'] = insample_mask
        
        return windows_batch
    
    def _aggregate_shap_by_feature(self, shap_values: np.ndarray, 
                              feature_names: List[str], 
                              aggregate_method: str) -> Tuple[np.ndarray, List[str]]:
        """
        Aggregate SHAP values across lags and steps for each feature.
        
        Parameters:
        -----------
        shap_values : np.ndarray
            Original SHAP values [n_samples, n_features]
        feature_names : List[str]
            Original feature names with lag/step information
        aggregate_method : str
            Aggregation method: 'sum', 'mean', 'abs_sum', 'max_abs'
            
        Returns:
        --------
        Tuple[np.ndarray, List[str]]
            Aggregated SHAP values and corresponding feature names
        """
        # Group feature indices by base name (without lag/step/series suffix)
        base_features: Dict[str, List[int]] = {}
        for i, name in enumerate(feature_names):
            if '_lag_' in name:
                # Extract base name by removing '_lag_X' suffix (historical features)
                base_name = '_'.join(name.split('_')[:-2])
            elif '_step_' in name:
                # Extract base name by removing '_step_X' suffix (future exogenous features)
                base_name = '_'.join(name.split('_')[:-2])
            elif '_series_' in name and '_static' in name:
                # Extract base name by removing '_series_X_static' suffix (multivariate static features)
                parts = name.split('_')
                # Find the index of 'series' and remove everything from there
                if 'series' in parts:
                    series_idx = parts.index('series')
                    base_name = '_'.join(parts[:series_idx])
                else:
                    base_name = name
            elif '_static' in name:
                # Extract base name by removing '_static' suffix (univariate static features)
                base_name = '_'.join(name.split('_')[:-1])
            else:
                # Feature without lag/step/static suffix (keep as is)
                base_name = name
            
            if base_name not in base_features:
                base_features[base_name] = []
            base_features[base_name].append(i)
        
        # Aggregate SHAP values for each base feature
        aggregated_shap_list = []
        aggregated_names = []
        
        for base_name, indices in base_features.items():
            feature_shap = shap_values[:, indices]  # [n_samples, n_lags_or_steps]
            
            if aggregate_method == 'sum':
                agg_shap = np.sum(feature_shap, axis=1, keepdims=True)
            elif aggregate_method == 'mean':
                agg_shap = np.mean(feature_shap, axis=1, keepdims=True)
            elif aggregate_method == 'abs_sum':
                agg_shap = np.sum(np.abs(feature_shap), axis=1, keepdims=True)
            elif aggregate_method == 'max_abs':
                max_indices = np.argmax(np.abs(feature_shap), axis=1)
                agg_shap = feature_shap[np.arange(len(feature_shap)), max_indices].reshape(-1, 1)
            else:
                raise ValueError(f"Unknown aggregation method: {aggregate_method}")
            
            aggregated_shap_list.append(agg_shap)
            aggregated_names.append(base_name)
        
        # Concatenate along feature dimension
        aggregated_shap_values = np.concatenate(aggregated_shap_list, axis=1)
        
        return aggregated_shap_values, aggregated_names

    def _create_shap_wrapper(self):
        """
        Create SHAP-compatible prediction wrapper for this model.
        """
        def predict_for_shap(X_flat: np.ndarray) -> np.ndarray:
            """SHAP prediction function with debugging."""
            # Set model to eval mode
            self.eval()
            
            # Parse input into model format
            windows_batch = self._parse_shap_input(X_flat)
            
            with torch.no_grad():
                try:
                    # Call model forward method with windows_batch dictionary
                    predictions = self.forward(windows_batch)
                    
                    # Handle different model output formats
                    if isinstance(predictions, tuple):
                        # Models like NBEATS return (backcast, forecast)
                        forecast = predictions[1]
                    else:
                        forecast = predictions
                    
                    if self.MULTIVARIATE:
                        # Multivariate models return [batch_size, horizon, n_series]
                        # Flatten to [batch_size, horizon * n_series] for SHAP
                        batch_size, horizon, n_series = forecast.shape
                        forecast = forecast.reshape(batch_size, horizon * n_series)
                    else:
                        # Univariate models: handle different output shapes
                        if len(forecast.shape) == 3 and forecast.shape[-1] > 1:
                            forecast = forecast.mean(dim=-1)  # Average across output dimensions
                        elif len(forecast.shape) == 3:
                            forecast = forecast.squeeze(-1)   # Remove single output dimension
                    
                    # Return numpy array
                    return forecast.cpu().numpy()
                    
                except Exception as e:
                    raise e
        
        return predict_for_shap

    def explain_prediction(self, 
                    input_df: pd.DataFrame,
                    X_df: Optional[pd.DataFrame] = None,
                    static_df: Optional[pd.DataFrame] = None,
                    background_size: int = 100,
                    target_samples: Optional[int] = 10,
                    explainer_type: str = 'kernel',
                    aggregate_lags: Optional[str] = None) -> Dict[str, Any]:
        """
        Generate SHAP explanations for model predictions using specific input data with temporal windowing.
        """
        try:
            import shap
            import pandas as pd
            import torch
            from neuralforecast.tsdataset import TimeSeriesDataset
            from torch.utils.data import DataLoader
        except ImportError:
            raise ImportError("SHAP library is required for explainability. Install with: pip install shap")
        
        # Custom collate function to handle pandas Index objects - DEFINE FIRST
        def custom_collate_fn(batch):
            """Custom collate function that handles pandas Index objects"""
            if len(batch) == 0:
                return {}
            
            first_sample = batch[0]
            collated = {}
            for key in first_sample.keys():
                if key in ['temporal_cols', 'static_cols']:
                    collated[key] = first_sample[key]
                elif key == 'y_idx':
                    collated[key] = first_sample[key]
                else:
                    values = [sample[key] for sample in batch]
                    if values[0] is not None:
                        collated[key] = torch.stack(values)
                    else:
                        collated[key] = None
            return collated
        
        # Validate input dataframe has required columns
        required_cols = ['unique_id', 'ds', 'y']
        missing_cols = [col for col in required_cols if col not in input_df.columns]
        if missing_cols:
            raise ValueError(f"input_df missing required columns: {missing_cols}")
        
        print("DEBUG: Input validation passed")
        print(f"DEBUG: input_df shape: {input_df.shape}")
        print(f"DEBUG: input_df unique_ids: {input_df['unique_id'].unique()}")
        
        # Check if we have sufficient historical data
        min_required_periods = self.input_size
        series_lengths = input_df.groupby('unique_id').size()
        insufficient_series = series_lengths[series_lengths < min_required_periods]
        if len(insufficient_series) > 0:
            raise ValueError(f"Some series have insufficient history. Need at least {min_required_periods} periods.")
        
        print("DEBUG: Historical data validation passed")
        
        # Validate exogenous features alignment
        has_hist_exog = (hasattr(self, 'EXOGENOUS_HIST') and self.EXOGENOUS_HIST and 
                        hasattr(self, 'hist_exog_list') and self.hist_exog_list)
        has_futr_exog = (hasattr(self, 'EXOGENOUS_FUTR') and self.EXOGENOUS_FUTR and 
                        hasattr(self, 'futr_exog_list') and self.futr_exog_list)
        has_stat_exog = (hasattr(self, 'EXOGENOUS_STAT') and self.EXOGENOUS_STAT and 
                        hasattr(self, 'stat_exog_list') and self.stat_exog_list)
        
        print(f"DEBUG: Exogenous feature flags - hist: {has_hist_exog}, futr: {has_futr_exog}, stat: {has_stat_exog}")
        
        # Validate exogenous features
        if has_hist_exog:
            missing_hist_exog = [col for col in self.hist_exog_list if col not in input_df.columns]
            if missing_hist_exog:
                raise ValueError(f"Model expects historical exogenous features but they're missing from input_df: {missing_hist_exog}")
        
        if has_futr_exog:
            if X_df is None:
                raise ValueError("Model expects future exogenous features but X_df is None")
            missing_futr_exog = [col for col in self.futr_exog_list if col not in X_df.columns]
            if missing_futr_exog:
                raise ValueError(f"Model expects future exogenous features but they're missing from X_df: {missing_futr_exog}")
            if 'unique_id' not in X_df.columns or 'ds' not in X_df.columns:
                raise ValueError("X_df must contain 'unique_id' and 'ds' columns")
        
        if has_stat_exog:
            if static_df is None:
                raise ValueError("Model expects static exogenous features but static_df is None")
            missing_stat_exog = [col for col in self.stat_exog_list if col not in static_df.columns]
            if missing_stat_exog:
                raise ValueError(f"Model expects static exogenous features but they're missing from static_df: {missing_stat_exog}")
            if 'unique_id' not in static_df.columns:
                raise ValueError("static_df must contain 'unique_id' column")
        
        try:
            # Always use temporal windowing - it's the right approach for time series
            print("DEBUG: Using temporal windowing approach for all feature types")
            
            horizon = getattr(self, 'h', 1)
            windows_list = []
            
            # Group by series to handle each one separately  
            for series_id, series_data in input_df.groupby('unique_id'):
                series_data = series_data.sort_values('ds').reset_index(drop=True)
                series_length = len(series_data)
                
                print(f"DEBUG: Creating windows for series {series_id}, length {series_length}")
                
            # Group by series to handle each one separately  
            for series_id, series_data in input_df.groupby('unique_id'):
                series_data = series_data.sort_values('ds').reset_index(drop=True)
                series_length = len(series_data)
                
                print(f"DEBUG: Creating windows for series {series_id}, length {series_length}")
                
                # For background windows: we need different amounts of data based on whether future exog is used
                if has_futr_exog and X_df is not None:
                    # With future exog: background windows need input_size + horizon to create historical "future" exog
                    min_background_data = self.input_size + horizon
                    print(f"DEBUG: With future exog - background windows need {min_background_data} periods each")
                else:
                    # Without future exog: background windows only need input_size periods
                    min_background_data = self.input_size
                    print(f"DEBUG: No future exog - background windows need {min_background_data} periods each")
                
                # Calculate how many background windows we can create from historical data
                # Leave the last input_size periods for the target window
                available_for_background = series_length - self.input_size
                max_possible_background = available_for_background - min_background_data + 1
                max_background_windows = min(background_size, max_possible_background)
                
                if max_background_windows < 1:
                    print(f"DEBUG: Series {series_id} has insufficient data for background windows")
                    print(f"DEBUG: Need {min_background_data} + {self.input_size} = {min_background_data + self.input_size} total periods")
                    print(f"DEBUG: Have {series_length} periods")
                    continue
                
                print(f"DEBUG: Can create {max_background_windows} background windows + 1 target window")
                
                # Get future exogenous data for this series if available
                series_X_df = None
                if X_df is not None and has_futr_exog:
                    series_X_df = X_df[X_df['unique_id'] == series_id].sort_values('ds').reset_index(drop=True)
                    print(f"DEBUG: Found {len(series_X_df)} future exog periods for series {series_id}")
                
                # Create BACKGROUND windows from historical periods
                for window_idx in range(max_background_windows):
                    start_idx = window_idx
                    hist_end_idx = start_idx + self.input_size
                    
                    # Get historical data for this background window
                    historical_data = series_data.iloc[start_idx:hist_end_idx].copy()
                    
                    # Start with the historical data
                    window_data = historical_data.copy()
                    
                    # IMPORTANT: If model expects future exog, background windows MUST have it too!
                    if series_X_df is not None and has_futr_exog:
                        print(f"DEBUG: Background window {window_idx} - adding historical future exog to match model expectations")
                        
                        # For background windows: use subsequent historical periods as "future" exog
                        future_start_idx = hist_end_idx
                        future_end_idx = future_start_idx + horizon
                        
                        if future_end_idx <= series_length:
                            # Use actual historical data as "future" exog for this background window
                            historical_future_data = series_data.iloc[future_start_idx:future_end_idx].copy()
                            
                            if self.__class__.__name__ == 'NHITS':
                                # NHITS needs input_size + horizon periods for future exog
                                # Use last input_size from historical + horizon from future
                                hist_futr_start = max(0, hist_end_idx - self.input_size)
                                hist_futr_data = series_data.iloc[hist_futr_start:hist_end_idx]
                                
                                # Combine historical + future periods
                                combined_bg_futr = pd.concat([hist_futr_data, historical_future_data], ignore_index=True)
                            else:
                                # Other models just need horizon periods
                                combined_bg_futr = historical_future_data
                            
                            # Extract only the future exogenous columns
                            futr_exog_data = combined_bg_futr[['unique_id', 'ds'] + self.futr_exog_list].copy()
                            
                            # Set y to 0 for future periods
                            futr_exog_data['y'] = 0.0
                            
                            # Add historical exog columns with 0 values
                            if has_hist_exog:
                                for hist_col in self.hist_exog_list:
                                    futr_exog_data[hist_col] = 0.0
                            
                            # Combine historical + historical "future" exog data
                            window_data = pd.concat([historical_data, futr_exog_data], ignore_index=True, sort=False)
                            
                            print(f"DEBUG: Background window {window_idx} using {len(futr_exog_data)} periods of historical future exog")
                        else:
                            print(f"DEBUG: Background window {window_idx} - insufficient data for future exog, skipping")
                            continue  # Skip this background window if we can't create proper future exog
                    
                    # Create unique identifier for each background window
                    window_data['unique_id'] = f"{series_id}_bg_{window_idx}"
                    window_data['window_idx'] = window_idx
                    window_data['original_series_id'] = series_id
                    window_data['is_target'] = False
                    
                    windows_list.append(window_data)
                    
                    print(f"DEBUG: Created background window {window_idx} for series {series_id}: "
                        f"dates {window_data['ds'].iloc[0]} to {window_data['ds'].iloc[-1]}, "
                        f"total periods: {len(window_data)}")
                
                # Create TARGET window - the actual prediction scenario
                # Use the most recent input_size periods from training data
                target_start_idx = series_length - self.input_size
                target_historical_data = series_data.iloc[target_start_idx:].copy()
                
                target_window_data = target_historical_data.copy()
                
                # Add future exogenous data if provided (this creates the actual prediction scenario)
                if series_X_df is not None and has_futr_exog:
                    print("DEBUG: Target window - adding actual future exog data from X_df")
                    
                    # For NHITS: need input_size + horizon future exog periods
                    if self.__class__.__name__ == 'NHITS':
                        # NHITS needs both historical and future periods for future exog
                        # Use last input_size periods from historical + all future exog data
                        hist_futr_start = max(0, series_length - self.input_size)
                        hist_futr_data = series_data.iloc[hist_futr_start:series_length]
                        
                        # Use the actual future exog data from X_df
                        actual_futr_data = series_X_df.copy()
                        
                        # Combine them for future exog
                        futr_exog_data = pd.concat([hist_futr_data[['unique_id', 'ds'] + self.futr_exog_list], 
                                                actual_futr_data[['unique_id', 'ds'] + self.futr_exog_list]], 
                                                ignore_index=True)
                    else:
                        # Other models just need actual future data
                        futr_exog_data = series_X_df[['unique_id', 'ds'] + self.futr_exog_list].copy()
                    
                    # Set y to 0 for future periods
                    futr_exog_data['y'] = 0.0
                    
                    # Add historical exog columns with 0 values for future periods
                    if has_hist_exog:
                        for hist_col in self.hist_exog_list:
                            futr_exog_data[hist_col] = 0.0
                    
                    print(f"DEBUG: Target window using {len(futr_exog_data)} periods of actual future exog")
                    
                    # Combine historical + actual future exog data for the target window
                    target_window_data = pd.concat([target_historical_data, futr_exog_data], ignore_index=True, sort=False)
                
                # Create unique identifier for target window
                target_window_data['unique_id'] = f"{series_id}_target"
                target_window_data['window_idx'] = max_background_windows
                target_window_data['original_series_id'] = series_id
                target_window_data['is_target'] = True
                
                windows_list.append(target_window_data)
                
                futr_exog_info = f"with {len(series_X_df)} future exog periods" if series_X_df is not None else "no future exog"
                print(f"DEBUG: Created target window for series {series_id}: "
                    f"dates {target_historical_data['ds'].iloc[0]} to {target_historical_data['ds'].iloc[-1]}, "
                    f"periods: {len(target_window_data)}, {futr_exog_info}")
                
                print(f"DEBUG: Total windows created for {series_id}: {max_background_windows} background + 1 target")
            
            if not windows_list:
                raise ValueError("No valid windows could be created from the provided data")
            
            # Combine all windows into a single dataframe
            all_windows_df = pd.concat(windows_list, ignore_index=True)
            
            # Fill any remaining NaN values
            all_windows_df = all_windows_df.fillna(0.0)
            
            print(f"DEBUG: Created {len(windows_list)} total windows")
            print(f"DEBUG: Background windows: {sum(1 for w in windows_list if not w['is_target'].iloc[0])}")
            print(f"DEBUG: Target windows: {sum(1 for w in windows_list if w['is_target'].iloc[0])}")
            
            # Prepare static_df for windows if needed
            window_static_df = None
            if static_df is not None:
                static_rows = []
                for window_data in windows_list:
                    original_series = window_data['original_series_id'].iloc[0]
                    window_unique_id = window_data['unique_id'].iloc[0]
                    
                    original_static = static_df[static_df['unique_id'] == original_series]
                    if len(original_static) > 0:
                        window_static = original_static.copy()
                        window_static['unique_id'] = window_unique_id
                        static_rows.append(window_static)
                
                if static_rows:
                    window_static_df = pd.concat(static_rows, ignore_index=True)
                    print(f"DEBUG: Created static data for {len(static_rows)} windows")
            
            # Create dataset using the standard method
            dataset, indices, dates, ds = TimeSeriesDataset.from_df(
                df=all_windows_df.drop(['window_idx', 'original_series_id', 'is_target'], axis=1),
                static_df=window_static_df,
                id_col='unique_id',
                time_col='ds',
                target_col='y'
            )
            
            print(f"DEBUG: TimeSeriesDataset created with {len(dataset)} samples")
            
            # Use temporal windowing sampling logic  
            dataloader = DataLoader(
                dataset, 
                batch_size=len(dataset), 
                shuffle=False,
                collate_fn=custom_collate_fn
            )
            batch = next(iter(dataloader))
            
            all_features = self._extract_target_features(batch)
            
            # Count background and target windows
            background_windows = sum(1 for w in windows_list if not w['is_target'].iloc[0])
            target_windows = sum(1 for w in windows_list if w['is_target'].iloc[0])
            
            background_indices = list(range(background_windows))
            target_indices = list(range(background_windows, background_windows + target_windows))
            
            if len(target_indices) == 0:
                raise ValueError("No target windows available for explanation.")
            
            background_data = all_features[background_indices]
            target_data = all_features[target_indices]
            
            print(f"DEBUG: Temporal windowing - Background: {background_data.shape}, Target: {target_data.shape}")
            
        except Exception as e:
            raise ValueError(f"Failed to create TimeSeriesDataset from provided data: {str(e)}")
        
        if len(target_data) == 0:
            raise ValueError("No target samples available for explanation.")
        
        # Create SHAP explainer
        prediction_wrapper = self._create_shap_wrapper()
        
        if explainer_type == 'kernel':
            explainer = shap.KernelExplainer(prediction_wrapper, background_data)
        else:
            raise ValueError(f"Explainer type '{explainer_type}' not supported yet")
        
        # Generate SHAP values
        shap_values = explainer.shap_values(target_data)
        
        # Get original feature names (with individual lags)
        original_feature_names = self._get_feature_names()
        
        # Apply lag aggregation if requested
        if aggregate_lags is not None:
            shap_values, feature_names = self._aggregate_shap_by_feature(
                shap_values, original_feature_names, aggregate_lags
            )
        else:
            feature_names = original_feature_names
        
        # Prepare results
        results = {
            'shap_values': shap_values,
            'feature_names': feature_names,
            'background_data': background_data,
            'target_data': target_data,
            'base_values': explainer.expected_value,
            'model_name': self.__class__.__name__,
            'model_alias': getattr(self, 'alias', None),
            'is_multivariate': self.MULTIVARIATE,
            'aggregate_lags': aggregate_lags,
            'original_feature_names': original_feature_names,
            'explained_periods': len(target_data),
            'background_periods': len(background_data),
            'total_series': len(input_df['unique_id'].unique()),
            'explanation_type': 'temporal_windows',
            'window_info': {
                'input_size': self.input_size,
                'horizon': getattr(self, 'h', 1),
                'background_samples': len(background_data),
                'target_samples': len(target_data),
                'has_futr_exog': has_futr_exog,
                'has_hist_exog': has_hist_exog,
                'has_stat_exog': has_stat_exog,
                'total_windows_created': len(windows_list)
            }
        }
        
        return results
    
    def on_fit_start(self):
        torch.manual_seed(self.random_seed)
        np.random.seed(self.random_seed)
        random.seed(self.random_seed)

    def configure_optimizers(self):
        if self.optimizer:
            optimizer_signature = inspect.signature(self.optimizer)
            optimizer_kwargs = deepcopy(self.optimizer_kwargs)
            if 'lr' in optimizer_signature.parameters:
                if 'lr' in optimizer_kwargs:
                    warnings.warn("ignoring learning rate passed in optimizer_kwargs, using the model's learning rate")
                optimizer_kwargs['lr'] = self.learning_rate
            optimizer = self.optimizer(params=self.parameters(), **optimizer_kwargs)
        else:
            if self.optimizer_kwargs:
                warnings.warn(
                    "ignoring optimizer_kwargs as the optimizer is not specified"
                )            
            optimizer = torch.optim.Adam(self.parameters(), lr=self.learning_rate)
        
        lr_scheduler = {'frequency': 1, 'interval': 'step'}
        if self.lr_scheduler:
            lr_scheduler_signature = inspect.signature(self.lr_scheduler)
            lr_scheduler_kwargs = deepcopy(self.lr_scheduler_kwargs)
            if 'optimizer' in lr_scheduler_signature.parameters:
                if 'optimizer' in lr_scheduler_kwargs:
                    warnings.warn("ignoring optimizer passed in lr_scheduler_kwargs, using the model's optimizer")
                    del lr_scheduler_kwargs['optimizer']
            lr_scheduler['scheduler'] = self.lr_scheduler(optimizer=optimizer, **lr_scheduler_kwargs)
        else:
            if self.lr_scheduler_kwargs:
                warnings.warn(
                    "ignoring lr_scheduler_kwargs as the lr_scheduler is not specified"
                )            
            lr_scheduler['scheduler'] = torch.optim.lr_scheduler.StepLR(
                optimizer=optimizer, step_size=self.lr_decay_steps, gamma=0.5
            )
        return {'optimizer': optimizer, 'lr_scheduler': lr_scheduler}

    def get_test_size(self):
        return self.test_size

    def set_test_size(self, test_size):
        self.test_size = test_size

    def on_validation_epoch_end(self):
        if self.val_size == 0:
            return
        losses = torch.stack(self.validation_step_outputs)
        avg_loss = losses.mean().detach().item()
        self.log(
            "ptl/val_loss",
            avg_loss,
            batch_size=losses.size(0),
            sync_dist=True,
        )
        self.valid_trajectories.append((self.global_step, avg_loss))
        self.validation_step_outputs.clear() # free memory (compute `avg_loss` per epoch)

    def save(self, path):
        with fsspec.open(path, 'wb') as f:
            torch.save(
                {'hyper_parameters': self.hparams, 'state_dict': self.state_dict()},
                f,
            )

    @classmethod
    def load(cls, path, **kwargs):
        if "weights_only" in inspect.signature(torch.load).parameters:
            kwargs["weights_only"] = False
        with fsspec.open(path, 'rb') as f, warnings.catch_warnings():
            # ignore possible warnings about weights_only=False
            warnings.filterwarnings('ignore', category=FutureWarning)
            content = torch.load(f, **kwargs)
        with _disable_torch_init():
            model = cls(**content['hyper_parameters']) 
        if "assign" in inspect.signature(model.load_state_dict).parameters:
            model.load_state_dict(content["state_dict"], strict=True, assign=True)
        else:  # pytorch<2.1
            model.load_state_dict(content["state_dict"], strict=True)
        return model

    def _create_windows(self, batch, step):
        # Parse common data
        window_size = self.input_size + self.h
        temporal_cols = batch['temporal_cols']
        temporal = batch['temporal']                

        if step == 'train':
            if self.val_size + self.test_size > 0:
                cutoff = -self.val_size - self.test_size
                temporal = temporal[:, :, :cutoff]

            temporal = self.padder_train(temporal)
            
            if temporal.shape[-1] < window_size:
                raise Exception('Time series is too short for training, consider setting a smaller input size or set start_padding_enabled=True')
            
            windows = temporal.unfold(dimension=-1, 
                                      size=window_size, 
                                      step=self.step_size)

            if self.MULTIVARIATE:
                # [n_series, C, Ws, L + h] -> [Ws, L + h, C, n_series]
                windows = windows.permute(2, 3, 1, 0)
            else:
                # [n_series, C, Ws, L + h] -> [Ws * n_series, L + h, C, 1]
                windows_per_serie = windows.shape[2]
                windows = windows.permute(0, 2, 3, 1)
                windows = windows.flatten(0, 1)
                windows = windows.unsqueeze(-1)

            # Sample and Available conditions
            available_idx = temporal_cols.get_loc('available_mask')           
            available_condition = windows[:, :self.input_size, available_idx]
            available_condition = torch.sum(available_condition, axis=(1, -1)) # Sum over time & series dimension
            final_condition = (available_condition > 0)
            
            if self.h > 0:
                sample_condition = windows[:, self.input_size:, available_idx]
                sample_condition = torch.sum(sample_condition, axis=(1, -1)) # Sum over time & series dimension
                final_condition = (sample_condition > 0) & (available_condition > 0)
            
            windows = windows[final_condition]

            # Parse Static data to match windows
            static = batch.get("static", None)
            static_cols = batch.get("static_cols", None)
            
            # Repeat static if univariate: [n_series, S] -> [Ws * n_series, S]
            if static is not None and not self.MULTIVARIATE:
                static = torch.repeat_interleave(static, 
                                    repeats=windows_per_serie, dim=0)
                static = static[final_condition]        

            # Protection of empty windows
            if final_condition.sum() == 0:
                raise Exception('No windows available for training')

            return windows, static, static_cols

        elif step in ['predict', 'val']:

            if step == 'predict':
                initial_input = temporal.shape[-1] - self.test_size
                if initial_input <= self.input_size: # There is not enough data to predict first timestamp
                    temporal = F.pad(temporal, pad=(self.input_size-initial_input, 0), mode="constant", value=0.0)
                predict_step_size = self.predict_step_size
                cutoff = - self.input_size - self.test_size
                temporal = temporal[:, :, cutoff:]

            elif step == 'val':
                predict_step_size = self.step_size
                cutoff = -self.input_size - self.val_size - self.test_size
                if self.test_size > 0:
                    temporal = batch['temporal'][:, :, cutoff:-self.test_size]
                else:
                    temporal = batch['temporal'][:, :, cutoff:]
                if temporal.shape[-1] < window_size:
                    initial_input = temporal.shape[-1] - self.val_size
                    temporal = F.pad(temporal, pad=(self.input_size-initial_input, 0), mode="constant", value=0.0)

            if (step=='predict') and (self.test_size==0) and (len(self.futr_exog_list)==0):
                temporal = F.pad(temporal, pad=(0, self.h), mode="constant", value=0.0)

            windows = temporal.unfold(dimension=-1,
                                      size=window_size,
                                      step=predict_step_size)

            static = batch.get('static', None)
            static_cols=batch.get('static_cols', None)

            if self.MULTIVARIATE:
                # [n_series, C, Ws, L + h] -> [Ws, L + h, C, n_series]
                windows = windows.permute(2, 3, 1, 0)
            else:
                # [n_series, C, Ws, L + h] -> [Ws * n_series, L + h, C, 1]
                windows_per_serie = windows.shape[2]
                windows = windows.permute(0, 2, 3, 1)
                windows = windows.flatten(0, 1)
                windows = windows.unsqueeze(-1)
                if static is not None:
                    static = torch.repeat_interleave(static, 
                                    repeats=windows_per_serie, dim=0)

            return windows, static, static_cols
        else:
            raise ValueError(f'Unknown step {step}') 

    def _normalization(self, windows, y_idx):
        # windows are already filtered by train/validation/test
        # from the `create_windows_method` nor leakage risk
        temporal = windows['temporal']                  # [Ws, L + h, C, n_series]
        temporal_cols = windows['temporal_cols'].copy() # [Ws, L + h, C, n_series]

        # To avoid leakage uses only the lags
        temporal_data_cols = self._get_temporal_exogenous_cols(temporal_cols=temporal_cols)
        temporal_idxs = get_indexer_raise_missing(temporal_cols, temporal_data_cols)
        temporal_idxs = np.append(y_idx, temporal_idxs)
        temporal_data = temporal[:, :, temporal_idxs] 
        temporal_mask = temporal[:, :, temporal_cols.get_loc('available_mask')].clone()
        if self.h > 0:
            temporal_mask[:, -self.h:] = 0.0

        # Normalize. self.scaler stores the shift and scale for inverse transform
        temporal_mask = temporal_mask.unsqueeze(2) # Add channel dimension for scaler.transform.
        temporal_data = self.scaler.transform(x=temporal_data, mask=temporal_mask)

        # Replace values in windows dict
        temporal[:, :, temporal_idxs] = temporal_data
        windows['temporal'] = temporal

        return windows

    def _inv_normalization(self, y_hat, y_idx):
        # Receives window predictions [Ws, h, output, n_series]
        # Broadcasts scale if necessary and inverts normalization
        add_channel_dim = y_hat.ndim > 3
        y_loc, y_scale = self._get_loc_scale(y_idx, add_channel_dim=add_channel_dim)
        y_hat = self.scaler.inverse_transform(z=y_hat, x_scale=y_scale, x_shift=y_loc)

        return y_hat
    
    def _sample_windows(self, windows_temporal, static, static_cols, temporal_cols, step, w_idxs=None):
        if step == 'train' and self.windows_batch_size is not None:
            n_windows = windows_temporal.shape[0]
            w_idxs = np.random.choice(n_windows, 
                                        size=self.windows_batch_size,
                                        replace=(n_windows < self.windows_batch_size))
        windows_sample = windows_temporal
        if w_idxs is not None:
            windows_sample = windows_temporal[w_idxs]
            
            if static is not None and not self.MULTIVARIATE:
                static = static[w_idxs]

        windows_batch = dict(temporal=windows_sample,
                                temporal_cols=temporal_cols,
                                static=static,
                                static_cols=static_cols)
        return windows_batch

    def _parse_windows(self, batch, windows):
        # windows: [Ws, L + h, C, n_series]

        # Filter insample lags from outsample horizon
        y_idx = batch['y_idx']
        mask_idx = batch['temporal_cols'].get_loc('available_mask')

        insample_y = windows['temporal'][:, :self.input_size, y_idx]
        insample_mask = windows['temporal'][:, :self.input_size, mask_idx]

        # Declare additional information
        outsample_y = None
        outsample_mask = None
        hist_exog = None
        futr_exog = None
        stat_exog = None

        if self.h > 0:
            outsample_y = windows['temporal'][:, self.input_size:, y_idx]
            outsample_mask = windows['temporal'][:, self.input_size:, mask_idx]

        # Recurrent models at t predict t+1, so we shift the input (insample_y) by one
        if self.RECURRENT:
            insample_y = torch.cat((insample_y, outsample_y[:, :-1]), dim=1)
            insample_mask = torch.cat((insample_mask, outsample_mask[:, :-1]), dim=1)
            self.maintain_state = False

        if len(self.hist_exog_list):
            hist_exog_idx = get_indexer_raise_missing(windows['temporal_cols'], self.hist_exog_list)
            if self.RECURRENT:
                hist_exog = windows['temporal'][:, :, hist_exog_idx]
                hist_exog[:, self.input_size:] = 0.0
                hist_exog = hist_exog[:, 1:]
            else:
                hist_exog = windows['temporal'][:, :self.input_size, hist_exog_idx]
            if not self.MULTIVARIATE:
                hist_exog = hist_exog.squeeze(-1)
            else:
                hist_exog = hist_exog.swapaxes(1, 2)

        if len(self.futr_exog_list):
            futr_exog_idx = get_indexer_raise_missing(windows['temporal_cols'], self.futr_exog_list)
            futr_exog = windows['temporal'][:, :, futr_exog_idx]
            if self.RECURRENT:
                futr_exog = futr_exog[:, 1:]
            if not self.MULTIVARIATE:
                futr_exog = futr_exog.squeeze(-1)
            else:
                futr_exog = futr_exog.swapaxes(1, 2)                

        if len(self.stat_exog_list):
            static_idx = get_indexer_raise_missing(windows['static_cols'], self.stat_exog_list)
            stat_exog = windows['static'][:, static_idx]

        # TODO: think a better way of removing insample_y features
        if self.exclude_insample_y:
            insample_y = insample_y * 0

        return insample_y, insample_mask, outsample_y, outsample_mask, \
               hist_exog, futr_exog, stat_exog     

    def _get_loc_scale(self, y_idx, add_channel_dim=False):
        # [B, L, C, n_series] -> [B, L, n_series]
        y_scale = self.scaler.x_scale[:, :, y_idx]
        y_loc = self.scaler.x_shift[:, :, y_idx]
        
        # [B, L, n_series] -> [B, L, n_series, 1]
        if add_channel_dim:
            y_scale = y_scale.unsqueeze(-1)
            y_loc = y_loc.unsqueeze(-1)

        return y_loc, y_scale

    def _compute_valid_loss(self, insample_y, outsample_y, output, outsample_mask, y_idx):
        if self.loss.is_distribution_output:
            y_loc, y_scale = self._get_loc_scale(y_idx)
            distr_args = self.loss.scale_decouple(output=output, loc=y_loc, scale=y_scale)
            if isinstance(self.valid_loss, (losses.sCRPS, losses.MQLoss, losses.HuberMQLoss)):
                _, _, quants  = self.loss.sample(distr_args=distr_args)            
                output = quants
            elif isinstance(self.valid_loss, losses.BasePointLoss):
                distr = self.loss.get_distribution(distr_args=distr_args)
                output = distr.mean

        # Validation Loss evaluation
        if self.valid_loss.is_distribution_output:
            valid_loss = self.valid_loss(y=outsample_y, distr_args=distr_args, mask=outsample_mask)
        else:
            output = self._inv_normalization(y_hat=output, y_idx=y_idx)
            valid_loss = self.valid_loss(y=outsample_y, y_hat=output, y_insample=insample_y, mask=outsample_mask)
        return valid_loss
    
    def _validate_step_recurrent_batch(self, insample_y, insample_mask, futr_exog, hist_exog, stat_exog, y_idx):
        # Remember state in network and set horizon to 1
        self.rnn_state = None
        self.maintain_state = True
        self.h = 1

        # Initialize results array
        n_outputs = self.loss.outputsize_multiplier
        y_hat = torch.zeros((insample_y.shape[0],
                            self.horizon_backup,
                            self.n_series * n_outputs),
                            device=insample_y.device,
                            dtype=insample_y.dtype)

        # First step prediction
        tau = 0
        
        # Set exogenous
        hist_exog_current = None
        if self.hist_exog_size > 0:
            hist_exog_current = hist_exog[:, :self.input_size + tau]

        futr_exog_current = None
        if self.futr_exog_size > 0:
            futr_exog_current = futr_exog[:, :self.input_size + tau]

        # First forecast step
        y_hat[:, tau], insample_y = self._validate_step_recurrent_single(
                                                                insample_y=insample_y[:, :self.input_size + tau],
                                                                insample_mask=insample_mask[:, :self.input_size + tau],
                                                                hist_exog=hist_exog_current,
                                                                futr_exog=futr_exog_current,
                                                                stat_exog=stat_exog,
                                                                y_idx=y_idx,
                                                                )

        # Horizon prediction recursively
        for tau in range(1, self.horizon_backup):
            # Set exogenous
            if self.hist_exog_size > 0:
                hist_exog_current = hist_exog[:, self.input_size + tau - 1].unsqueeze(1)

            if self.futr_exog_size > 0:
                futr_exog_current = futr_exog[:, self.input_size + tau - 1].unsqueeze(1)
            
            y_hat[:, tau], insample_y = self._validate_step_recurrent_single(
                                                                insample_y=insample_y,
                                                                insample_mask=None,
                                                                hist_exog=hist_exog_current,
                                                                futr_exog=futr_exog_current,
                                                                stat_exog=stat_exog,
                                                                y_idx = y_idx,
                                                                )
        
        # Reset state and horizon
        self.maintain_state = False
        self.rnn_state = None
        self.h = self.horizon_backup

        return y_hat   

    def _validate_step_recurrent_single(self, insample_y, insample_mask, hist_exog, futr_exog, stat_exog, y_idx):
        # Input sequence
        windows_batch = dict(insample_y=insample_y,                 # [Ws, L, n_series]
                        insample_mask=insample_mask,                # [Ws, L, n_series]
                        futr_exog=futr_exog,                        # univariate: [Ws, L, F]; multivariate: [Ws, F, L, n_series]
                        hist_exog=hist_exog,                        # univariate: [Ws, L, X]; multivariate: [Ws, X, L, n_series]
                        stat_exog=stat_exog)                        # univariate: [Ws, S]; multivariate: [n_series, S]

        # Model Predictions
        output_batch_unmapped = self(windows_batch)
        output_batch = self.loss.domain_map(output_batch_unmapped)
        
        # Inverse normalization and sampling
        if self.loss.is_distribution_output:
            # Sample distribution
            y_loc, y_scale = self._get_loc_scale(y_idx)
            distr_args = self.loss.scale_decouple(output=output_batch, loc=y_loc, scale=y_scale)
            # When validating, the output is the mean of the distribution which is an attribute
            distr = self.loss.get_distribution(distr_args=distr_args)

            # Scale back to feed back as input
            insample_y = self.scaler.scaler(distr.mean, y_loc, y_scale)
        else:
            # Todo: for now, we assume that in case of a BasePointLoss with ndim==4, the last dimension
            # contains a set of predictions for the target (e.g. MQLoss multiple quantiles), for which we use the 
            # mean as feedback signal for the recurrent predictions. A more precise way is to increase the
            # insample input size of the recurrent network by the number of outputs so that each output
            # can be fed back to a specific input channel. 
            if output_batch.ndim == 4:
                output_batch = output_batch.mean(dim=-1)

            insample_y = output_batch

        # Remove horizon dim: [B, 1, N * n_outputs] -> [B, N * n_outputs]
        y_hat = output_batch_unmapped.squeeze(1)
        return y_hat, insample_y

    def _predict_step_recurrent_batch(self, insample_y, insample_mask, futr_exog, hist_exog, stat_exog, y_idx):
        # Remember state in network and set horizon to 1
        self.rnn_state = None
        self.maintain_state = True
        self.h = 1

        # Initialize results array
        n_outputs = len(self.loss.output_names)
        y_hat = torch.zeros((insample_y.shape[0],
                            self.horizon_backup,
                            self.n_series,
                            n_outputs),
                            device=insample_y.device,
                            dtype=insample_y.dtype)

        # First step prediction
        tau = 0
        
        # Set exogenous
        hist_exog_current = None
        if self.hist_exog_size > 0:
            hist_exog_current = hist_exog[:, :self.input_size + tau]

        futr_exog_current = None
        if self.futr_exog_size > 0:
            futr_exog_current = futr_exog[:, :self.input_size + tau]

        # First forecast step
        y_hat[:, tau], insample_y = self._predict_step_recurrent_single(
                                                                insample_y=insample_y[:, :self.input_size + tau],
                                                                insample_mask=insample_mask[:, :self.input_size + tau],
                                                                hist_exog=hist_exog_current,
                                                                futr_exog=futr_exog_current,
                                                                stat_exog=stat_exog,
                                                                y_idx=y_idx,
                                                                )

        # Horizon prediction recursively
        for tau in range(1, self.horizon_backup):
            # Set exogenous
            if self.hist_exog_size > 0:
                hist_exog_current = hist_exog[:, self.input_size + tau - 1].unsqueeze(1)

            if self.futr_exog_size > 0:
                futr_exog_current = futr_exog[:, self.input_size + tau - 1].unsqueeze(1)
            
            y_hat[:, tau], insample_y = self._predict_step_recurrent_single(
                                                                insample_y=insample_y,
                                                                insample_mask=None,
                                                                hist_exog=hist_exog_current,
                                                                futr_exog=futr_exog_current,
                                                                stat_exog=stat_exog,
                                                                y_idx = y_idx,
                                                                )
        
        # Reset state and horizon
        self.maintain_state = False
        self.rnn_state = None
        self.h = self.horizon_backup

        # Squeeze for univariate case
        if not self.MULTIVARIATE:
            y_hat = y_hat.squeeze(2)

        return y_hat        

    def _predict_step_recurrent_single(self, insample_y, insample_mask, hist_exog, futr_exog, stat_exog, y_idx):
        # Input sequence
        windows_batch = dict(insample_y=insample_y,                 # [Ws, L, n_series]
                        insample_mask=insample_mask,                # [Ws, L, n_series]
                        futr_exog=futr_exog,                        # univariate: [Ws, L, F]; multivariate: [Ws, F, L, n_series]
                        hist_exog=hist_exog,                        # univariate: [Ws, L, X]; multivariate: [Ws, X, L, n_series]
                        stat_exog=stat_exog)                        # univariate: [Ws, S]; multivariate: [n_series, S]

        # Model Predictions
        output_batch_unmapped = self(windows_batch)
        output_batch = self.loss.domain_map(output_batch_unmapped)
        
        # Inverse normalization and sampling
        if self.loss.is_distribution_output:
            # Sample distribution
            y_loc, y_scale = self._get_loc_scale(y_idx)
            distr_args = self.loss.scale_decouple(output=output_batch, loc=y_loc, scale=y_scale)
            # When predicting, we need to sample to get the quantiles. The mean is an attribute.
            _, _, quants = self.loss.sample(distr_args=distr_args, num_samples=self.n_samples)
            mean = self.loss.distr_mean

            # Scale back to feed back as input
            insample_y = self.scaler.scaler(mean, y_loc, y_scale)
            
            # Save predictions
            y_hat = torch.concat((mean.unsqueeze(-1), quants), axis=-1)

            if self.loss.return_params:
                distr_args = torch.stack(distr_args, dim=-1)
                if distr_args.ndim > 4:
                    distr_args = distr_args.flatten(-2, -1)
                y_hat = torch.concat((y_hat, distr_args), axis=-1)
        else:
            # Todo: for now, we assume that in case of a BasePointLoss with ndim==4, the last dimension
            # contains a set of predictions for the target (e.g. MQLoss multiple quantiles), for which we use the 
            # mean as feedback signal for the recurrent predictions. A more precise way is to increase the
            # insample input size of the recurrent network by the number of outputs so that each output
            # can be fed back to a specific input channel. 
            if output_batch.ndim == 4:
                output_batch = output_batch.mean(dim=-1)

            insample_y = output_batch
            y_hat = self._inv_normalization(y_hat=output_batch, y_idx=y_idx)
            y_hat = y_hat.unsqueeze(-1)

        # Remove horizon dim: [B, 1, N, n_outputs] -> [B, N, n_outputs]
        y_hat = y_hat.squeeze(1)
        return y_hat, insample_y

    def _predict_step_direct_batch(self, insample_y, insample_mask, hist_exog, futr_exog, stat_exog, y_idx):
        windows_batch = dict(insample_y=insample_y,                 # [Ws, L, n_series]
                        insample_mask=insample_mask,                # [Ws, L, n_series]
                        futr_exog=futr_exog,                        # univariate: [Ws, L, F]; multivariate: [Ws, F, L, n_series]
                        hist_exog=hist_exog,                        # univariate: [Ws, L, X]; multivariate: [Ws, X, L, n_series]
                        stat_exog=stat_exog)                        # univariate: [Ws, S]; multivariate: [n_series, S]

        # Model Predictions
        output_batch = self(windows_batch)
        output_batch = self.loss.domain_map(output_batch)

        # Inverse normalization and sampling
        if self.loss.is_distribution_output:
            y_loc, y_scale = self._get_loc_scale(y_idx)
            distr_args = self.loss.scale_decouple(output=output_batch, loc=y_loc, scale=y_scale)
            _, sample_mean, quants = self.loss.sample(distr_args=distr_args)
            y_hat = torch.concat((sample_mean, quants), axis=-1)

            if self.loss.return_params:
                distr_args = torch.stack(distr_args, dim=-1)
                if distr_args.ndim > 4:
                    distr_args = distr_args.flatten(-2, -1)
                y_hat = torch.concat((y_hat, distr_args), axis=-1)                
        else:
             y_hat = self._inv_normalization(y_hat=output_batch, 
                                            y_idx=y_idx)

        return y_hat
            
    def training_step(self, batch, batch_idx):
        # Set horizon to h_train in case of recurrent model to speed up training
        if self.RECURRENT:
            self.h = self.h_train
        
        # windows: [Ws, L + h, C, n_series] or [Ws, L + h, C]
        y_idx = batch['y_idx']

        temporal_cols = batch['temporal_cols']
        windows_temporal, static, static_cols = self._create_windows(batch, step='train')
        windows = self._sample_windows(windows_temporal, static, static_cols, temporal_cols, step='train')
        original_outsample_y = torch.clone(windows['temporal'][:, self.input_size:, y_idx])
        windows = self._normalization(windows=windows, y_idx=y_idx)
        
        # Parse windows
        insample_y, insample_mask, outsample_y, outsample_mask, \
               hist_exog, futr_exog, stat_exog = self._parse_windows(batch, windows)

        windows_batch = dict(insample_y=insample_y,                 # [Ws, L, n_series]
                        insample_mask=insample_mask,                # [Ws, L, n_series]
                        futr_exog=futr_exog,                        # univariate: [Ws, L, F]; multivariate: [Ws, F, L, n_series]
                        hist_exog=hist_exog,                        # univariate: [Ws, L, X]; multivariate: [Ws, X, L, n_series]
                        stat_exog=stat_exog)                        # univariate: [Ws, S]; multivariate: [n_series, S]

        # Model Predictions
        output = self(windows_batch)
        output = self.loss.domain_map(output)
        
        if self.loss.is_distribution_output:
            y_loc, y_scale = self._get_loc_scale(y_idx)
            outsample_y = original_outsample_y
            distr_args = self.loss.scale_decouple(output=output, loc=y_loc, scale=y_scale)
            loss = self.loss(y=outsample_y, distr_args=distr_args, mask=outsample_mask)
        else:
            loss = self.loss(y=outsample_y, y_hat=output, y_insample=insample_y, mask=outsample_mask)

        if torch.isnan(loss):
            print('Model Parameters', self.hparams)
            print('insample_y', torch.isnan(insample_y).sum())
            print('outsample_y', torch.isnan(outsample_y).sum())
            raise Exception('Loss is NaN, training stopped.')

        train_loss_log = loss.detach().item()
        self.log(
            'train_loss',
            train_loss_log,
            batch_size=outsample_y.size(0),
            prog_bar=True,
            on_epoch=True,
        )
        self.train_trajectories.append((self.global_step, train_loss_log))

        self.h = self.horizon_backup

        return loss


    def validation_step(self, batch, batch_idx):
        if self.val_size == 0:
            return np.nan

        temporal_cols = batch['temporal_cols']
        windows_temporal, static, static_cols = self._create_windows(batch, step='val')
        n_windows = len(windows_temporal)
        y_idx = batch['y_idx']

        # Number of windows in batch
        windows_batch_size = self.inference_windows_batch_size
        if windows_batch_size < 0:
            windows_batch_size = n_windows
        n_batches = int(np.ceil(n_windows / windows_batch_size))

        valid_losses = []
        batch_sizes = []
        for i in range(n_batches):
            # Create and normalize windows [Ws, L + h, C, n_series]
            w_idxs = np.arange(i*windows_batch_size, 
                               min((i+1)*windows_batch_size, n_windows))
            windows = self._sample_windows(windows_temporal, static, static_cols, temporal_cols, step='val', w_idxs=w_idxs)
            original_outsample_y = torch.clone(windows['temporal'][:, self.input_size:, y_idx])

            windows = self._normalization(windows=windows, y_idx=y_idx)

            # Parse windows
            insample_y, insample_mask, _, outsample_mask, \
                hist_exog, futr_exog, stat_exog = self._parse_windows(batch, windows)

            if self.RECURRENT:
                output_batch = self._validate_step_recurrent_batch(insample_y=insample_y,
                                                           insample_mask=insample_mask,
                                                           futr_exog=futr_exog,
                                                           hist_exog=hist_exog,
                                                           stat_exog=stat_exog,
                                                           y_idx=y_idx)
            else:       
                windows_batch = dict(insample_y=insample_y,                 # [Ws, L, n_series]
                                insample_mask=insample_mask,                # [Ws, L, n_series]
                                futr_exog=futr_exog,                        # univariate: [Ws, L, F]; multivariate: [Ws, F, L, n_series]
                                hist_exog=hist_exog,                        # univariate: [Ws, L, X]; multivariate: [Ws, X, L, n_series]
                                stat_exog=stat_exog)                        # univariate: [Ws, S]; multivariate: [n_series, S]
                
                # Model Predictions
                output_batch = self(windows_batch)   

            output_batch = self.loss.domain_map(output_batch)
            valid_loss_batch = self._compute_valid_loss(insample_y=insample_y,
                                                        outsample_y=original_outsample_y,
                                                output=output_batch, 
                                                outsample_mask=outsample_mask,
                                                y_idx=batch['y_idx'])
            valid_losses.append(valid_loss_batch)
            batch_sizes.append(len(output_batch))
        
        valid_loss = torch.stack(valid_losses)
        batch_sizes = torch.tensor(batch_sizes, device=valid_loss.device)
        batch_size = torch.sum(batch_sizes)
        valid_loss = torch.sum(valid_loss * batch_sizes) / batch_size

        if torch.isnan(valid_loss):
            raise Exception('Loss is NaN, training stopped.')

        valid_loss_log = valid_loss.detach()
        self.log(
            'valid_loss',
            valid_loss_log.item(),
            batch_size=batch_size,
            prog_bar=True,
            on_epoch=True,
        )
        self.validation_step_outputs.append(valid_loss_log)
        return valid_loss

    def predict_step(self, batch, batch_idx):
        if self.RECURRENT:
            self.input_size = self.inference_input_size

        temporal_cols = batch['temporal_cols']
        windows_temporal, static, static_cols = self._create_windows(batch, step='predict')
        n_windows = len(windows_temporal)
        y_idx = batch['y_idx']

        # Number of windows in batch
        windows_batch_size = self.inference_windows_batch_size
        if windows_batch_size < 0:
            windows_batch_size = n_windows
        n_batches = int(np.ceil(n_windows / windows_batch_size))
        y_hats = []
        for i in range(n_batches):
            # Create and normalize windows [Ws, L+H, C]
            w_idxs = np.arange(i*windows_batch_size, 
                    min((i+1)*windows_batch_size, n_windows))
            windows = self._sample_windows(windows_temporal, static, static_cols, temporal_cols, step='predict', w_idxs=w_idxs)
            windows = self._normalization(windows=windows, y_idx=y_idx)

            # Parse windows
            insample_y, insample_mask, _, _, \
                hist_exog, futr_exog, stat_exog = self._parse_windows(batch, windows)

            if self.RECURRENT:                
                y_hat = self._predict_step_recurrent_batch(insample_y=insample_y,
                                                           insample_mask=insample_mask,
                                                           futr_exog=futr_exog,
                                                           hist_exog=hist_exog,
                                                           stat_exog=stat_exog,
                                                           y_idx=y_idx)
            else:
                y_hat = self._predict_step_direct_batch(insample_y=insample_y,
                                                           insample_mask=insample_mask,
                                                           futr_exog=futr_exog,
                                                           hist_exog=hist_exog,
                                                           stat_exog=stat_exog,
                                                           y_idx=y_idx)                


            y_hats.append(y_hat)
        y_hat = torch.cat(y_hats, dim=0)
        self.input_size = self.input_size_backup

        return y_hat
    
    def fit(self, dataset, val_size=0, test_size=0, random_seed=None, distributed_config=None):
        """ Fit.

        The `fit` method, optimizes the neural network's weights using the
        initialization parameters (`learning_rate`, `windows_batch_size`, ...)
        and the `loss` function as defined during the initialization. 
        Within `fit` we use a PyTorch Lightning `Trainer` that
        inherits the initialization's `self.trainer_kwargs`, to customize
        its inputs, see [PL's trainer arguments](https://pytorch-lightning.readthedocs.io/en/stable/api/pytorch_lightning.trainer.trainer.Trainer.html?highlight=trainer).

        The method is designed to be compatible with SKLearn-like classes
        and in particular to be compatible with the StatsForecast library.

        By default the `model` is not saving training checkpoints to protect 
        disk memory, to get them change `enable_checkpointing=True` in `__init__`.

        **Parameters:**<br>
        `dataset`: NeuralForecast's `TimeSeriesDataset`, see [documentation](https://nixtla.github.io/neuralforecast/tsdataset.html).<br>
        `val_size`: int, validation size for temporal cross-validation.<br>
        `random_seed`: int=None, random_seed for pytorch initializer and numpy generators, overwrites model.__init__'s.<br>
        `test_size`: int, test size for temporal cross-validation.<br>
        """
        return self._fit(
            dataset=dataset,
            batch_size=self.batch_size,
            valid_batch_size=self.valid_batch_size,
            val_size=val_size,
            test_size=test_size,
            random_seed=random_seed,
            distributed_config=distributed_config,
        )

    def predict(self, dataset, test_size=None, step_size=1,
                random_seed=None, quantiles=None, **data_module_kwargs):
        """ Predict.

        Neural network prediction with PL's `Trainer` execution of `predict_step`.

        **Parameters:**<br>
        `dataset`: NeuralForecast's `TimeSeriesDataset`, see [documentation](https://nixtla.github.io/neuralforecast/tsdataset.html).<br>
        `test_size`: int=None, test size for temporal cross-validation.<br>
        `step_size`: int=1, Step size between each window.<br>
        `random_seed`: int=None, random_seed for pytorch initializer and numpy generators, overwrites model.__init__'s.<br>
        `quantiles`: list of floats, optional (default=None), target quantiles to predict. <br>
        `**data_module_kwargs`: PL's TimeSeriesDataModule args, see [documentation](https://pytorch-lightning.readthedocs.io/en/1.6.1/extensions/datamodules.html#using-a-datamodule).
        """
        self._check_exog(dataset)
        self._restart_seed(random_seed)
        if "quantile" in data_module_kwargs:
            warnings.warn("The 'quantile' argument will be deprecated, use 'quantiles' instead.")
            if quantiles is not None:
                raise ValueError("You can't specify quantile and quantiles.")
            quantiles = [data_module_kwargs.pop("quantile")]
        self._set_quantiles(quantiles)

        self.predict_step_size = step_size
        self.decompose_forecast = False
        datamodule = TimeSeriesDataModule(dataset=dataset,
                                          valid_batch_size=self.valid_batch_size,
                                          **data_module_kwargs)

        # Protect when case of multiple gpu. PL does not support return preds with multiple gpu.
        pred_trainer_kwargs = self.trainer_kwargs.copy()
        if (pred_trainer_kwargs.get('accelerator', None) == "gpu") and (torch.cuda.device_count() > 1):
            pred_trainer_kwargs['devices'] = [0]

        trainer = pl.Trainer(**pred_trainer_kwargs)
        fcsts = trainer.predict(self, datamodule=datamodule)        
        fcsts = torch.vstack(fcsts)

        if self.MULTIVARIATE:
            # [B, h, n_series (, Q)] -> [n_series, B, h (, Q)]
            fcsts = fcsts.swapaxes(0, 2)
            fcsts = fcsts.swapaxes(1, 2)

        fcsts = tensor_to_numpy(fcsts).flatten()
        fcsts = fcsts.reshape(-1, len(self.loss.output_names))
        return fcsts

    def decompose(self, dataset, step_size=1, random_seed=None, quantiles=None, **data_module_kwargs):
        """ Decompose Predictions.

        Decompose the predictions through the network's layers.
        Available methods are `ESRNN`, `NHITS`, `NBEATS`, and `NBEATSx`.

        **Parameters:**<br>
        `dataset`: NeuralForecast's `TimeSeriesDataset`, see [documentation here](https://nixtla.github.io/neuralforecast/tsdataset.html).<br>
        `step_size`: int=1, step size between each window of temporal data.<br>
        `quantiles`: list of floats, optional (default=None), target quantiles to predict. <br>
        `**data_module_kwargs`: PL's TimeSeriesDataModule args, see [documentation](https://pytorch-lightning.readthedocs.io/en/1.6.1/extensions/datamodules.html#using-a-datamodule).
        """
        # Restart random seed
        if random_seed is None:
            random_seed = self.random_seed
        torch.manual_seed(random_seed)
        self._set_quantiles(quantiles)

        self.predict_step_size = step_size
        self.decompose_forecast = True
        datamodule = TimeSeriesDataModule(dataset=dataset,
                                          valid_batch_size=self.valid_batch_size,
                                          **data_module_kwargs)
        trainer = pl.Trainer(**self.trainer_kwargs)
        fcsts = trainer.predict(self, datamodule=datamodule)
        self.decompose_forecast = False # Default decomposition back to false
        fcsts = torch.vstack(fcsts)
        return tensor_to_numpy(fcsts)        